# Tensorboard with One-layer Neural Network

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

## function for variables

In [2]:
def init_weights(shape, name):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.01), name=name)

## function for model construction

## Sharing variables using tf.variable_scope
###  tensorboard 활용을 잘하기위해서 layer name을 쓴다.

In [3]:
def model(X, w_h, w_h2):
    with tf.variable_scope("layer1"):
        h = tf.nn.relu(tf.matmul(X, w_h))
    with tf.variable_scope("layer2"):        
        h2 = tf.nn.relu(tf.matmul(h, w_h2))
    with tf.variable_scope("layer3"):        
        return tf.matmul(h2, w_o)

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)
X_trn, Y_trn, X_test, Y_test = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## make placeholder for inputs and labels

In [5]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int32, [None])

## weight variable 와 Model 만들기
### 첫번째 hidden layer은 input을 784 / 2 = 392 차원으로 보낸다.
### 두번째 hidden layer은 392차원을 다시 392차원으로 보낸다.
### output layer에서는 이를 label의 차원인 10차원으로 보낸다.

In [6]:
hidden_dim = 392
w_h = init_weights([784, hidden_dim], name = "w_h")
w_h2 = init_weights([hidden_dim, hidden_dim], name = "w_h2")
w_o = init_weights([hidden_dim, 10], name = "w_o")

py_x = model(X, w_h, w_h2)

## Weight variable에 대해서 hitogram summary 만들기

In [7]:
tf.histogram_summary("w_h_summ", w_h)
tf.histogram_summary("w_h2_summ", w_h2)
tf.histogram_summary("w_o_summ", w_o)

<tf.Tensor 'HistogramSummary_2:0' shape=() dtype=string>

## loss function

In [8]:
with tf.variable_scope("loss"):
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(py_x, Y))
    tf.scalar_summary("loss", loss)

## Training operator
#### tf.train.AdagradOptimizer

In [9]:
train_op = tf.train.AdagradOptimizer(learning_rate=0.05).minimize(loss)

## Accuracy 계산
### tensorflow를 통해서 accuracy를 계산

In [10]:
with tf.variable_scope("accuracy"):
    correct = tf.nn.in_top_k(py_x, Y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    tf.scalar_summary("accuracy", accuracy)

## Session initialize 및 training

## SummaryWriter를 통해서 event file을 만들어서 tensorboard를 이용할 수 있게 한다.
## merge_all_summaries() 를 이용하여 여러 summary들을 한번에 계산한다.

In [11]:
batch_size = 128
with tf.Session() as sess:    
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(X_trn), batch_size), range(batch_size, len(X_trn)+1, batch_size)):
            sess.run(train_op, feed_dict={X: X_trn[start:end], Y: Y_trn[start:end]})
        summary, test_acc = sess.run([merged, accuracy], feed_dict={X: X_test, Y: Y_test})
        writer.add_summary(summary, i) #i 번째 step
        print("epoch: {} test accuracy {:0.4f}".format(i, test_acc))

epoch: 0 test accuracy 0.8761
epoch: 1 test accuracy 0.9204
epoch: 2 test accuracy 0.9463
epoch: 3 test accuracy 0.9608
epoch: 4 test accuracy 0.9666
epoch: 5 test accuracy 0.9708
epoch: 6 test accuracy 0.9726
epoch: 7 test accuracy 0.9742
epoch: 8 test accuracy 0.9748
epoch: 9 test accuracy 0.9753
epoch: 10 test accuracy 0.9753
epoch: 11 test accuracy 0.9756
epoch: 12 test accuracy 0.9763
epoch: 13 test accuracy 0.9767
epoch: 14 test accuracy 0.9774
epoch: 15 test accuracy 0.9774
epoch: 16 test accuracy 0.9776
epoch: 17 test accuracy 0.9785
epoch: 18 test accuracy 0.9790
epoch: 19 test accuracy 0.9788
epoch: 20 test accuracy 0.9787
epoch: 21 test accuracy 0.9787
epoch: 22 test accuracy 0.9789
epoch: 23 test accuracy 0.9792
epoch: 24 test accuracy 0.9794
epoch: 25 test accuracy 0.9796
epoch: 26 test accuracy 0.9794
epoch: 27 test accuracy 0.9793
epoch: 28 test accuracy 0.9792
epoch: 29 test accuracy 0.9793
epoch: 30 test accuracy 0.9794
epoch: 31 test accuracy 0.9795
epoch: 32 test acc

## terminal 을 이용하여
## tensorboard --logdir=/path/to/log-directory
## 실행